In [1]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='z2t0_flSWgJMKAYjjpbV0-e5fyqfooI_fVZrBTPLG7og',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'kidneydiseaseprediction-donotdelete-pr-xasqa3izva58lu'
object_key = 'ProcessedKidney_SelectedFeatureTrainData.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,bp,sg,pc,bu,hemo,pcv,rc,htn,dm,pe,classification
0,80.0,1.020,0,36.0,15.4,44.0,5.2,1,1,0,1
1,50.0,1.020,0,18.0,11.3,38.0,4.8,0,0,0,1
2,80.0,1.010,0,53.0,9.6,31.0,4.8,0,1,0,1
3,70.0,1.005,1,56.0,11.2,32.0,3.9,1,0,1,1
4,80.0,1.010,0,26.0,11.6,35.0,4.6,0,0,0,1


In [3]:
X = df.iloc[: , :-1]
Y = df["classification"]

In [4]:
X.head()

,bp,sg,pc,bu,hemo,pcv,rc,htn,dm,pe
0,80.0,1.020,0,36.0,15.4,44.0,5.2,1,1,0
1,50.0,1.020,0,18.0,11.3,38.0,4.8,0,0,0
2,80.0,1.010,0,53.0,9.6,31.0,4.8,0,1,0
3,70.0,1.005,1,56.0,11.2,32.0,3.9,1,0,1
4,80.0,1.010,0,26.0,11.6,35.0,4.6,0,0,0


In [5]:
classifier = ExtraTreesClassifier(n_estimators = 5,criterion ='entropy', max_features = 2)

In [6]:
classifier.fit(X , Y)

ExtraTreesClassifier(criterion='entropy', max_features=2, n_estimators=5)

In [9]:
classifier.predict(["80.0	1.020	0	49.0	15.7	47.0	4.9	0	0	0".split()])

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but ExtraTreesClassifier was fitted with feature names
  warnings.warn(


array([0])

deployment

In [10]:
!pip install -U ibm-watson-machine-learning

In [11]:
from ibm_watson_machine_learning import APIClient

In [12]:
wml_credentials = {
    "apikey":"Im5Owvbo4HbMZgEjLP9zMlCv-d_kwEGOkQgNoDH5C38M",
    "url":"https://eu-de.ml.cloud.ibm.com"
}

In [13]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----------  ------------------------
ID                                    NAME         CREATED
566ede0c-aea9-467b-af05-7f9529d6aa21  kidney_pred  2022-11-05T13:41:33.148Z
------------------------------------  -----------  ------------------------


In [14]:
space_id = "566ede0c-aea9-467b-af05-7f9529d6aa21"

In [15]:
wml_client.set.default_space(space_id)

'SUCCESS'

In [16]:
MODEL_NAME = 'DemoModel'
DEPLOYMENT_NAME = 'demo_deploy'
DEMO_MODEL = classifier

In [17]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [18]:
# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [19]:
#Save model
model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=X, 
    training_target=Y
)

In [20]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'classification',
  'schemas': {'input': [{'fields': [{'name': 'bp', 'type': 'float64'},
      {'name': 'sg', 'type': 'float64'},
      {'name': 'pc', 'type': 'int64'},
      {'name': 'bu', 'type': 'float64'},
      {'name': 'hemo', 'type': 'float64'},
      {'name': 'pcv', 'type': 'float64'},
      {'name': 'rc', 'type': 'float64'},
      {'name': 'htn', 'type': 'int64'},
      {'name': 'dm', 'type': 'int64'},
      {'name': 'pe', 'type': 'int64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-19T04:59:42.144Z',
  'id': '2b94b0a9-fafa-49c4-a189-4064bd7f2848',
  'modified_at': '2022-11-19T04:59:46.210Z',
  'name': 'DemoModel',
  'owner': 'IBMid-6630043OFC',
  'resource_key': 'bfb57ea6-8dce-4ee6-a2b7-7cdeb2a87531',
  'space_id': '566ede0c-ae